In [ ]:
# Check GPU availability
import torch
print(f"🔍 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU device: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected. Please enable GPU in Runtime → Change runtime type")

# Install required packages
print("\n📦 Installing dependencies...")
%pip install -q gradio>=4.0.0
%pip install -q opencv-python-headless
%pip install -q timm
%pip install -q matplotlib

print("✅ Dependencies installed!")


In [ ]:
import os
import sys

# Clone the repository if not already present
if not os.path.exists('ml-depth-pro'):
    print("📥 Cloning Apple's Depth Pro repository...")
    !git clone https://github.com/apple/ml-depth-pro.git
else:
    print("✅ Repository already exists")

# Change to the repository directory
os.chdir('ml-depth-pro')
print(f"📁 Current directory: {os.getcwd()}")

# Add to Python path
sys.path.insert(0, './src')
sys.path.insert(0, '.')

print("✅ Repository setup complete!")


In [ ]:
# Download the pre-trained model weights
import os
import urllib.request
from tqdm import tqdm

def download_with_progress(url, filename):
    """Download file with progress bar"""
    class TqdmUpTo(tqdm):
        def update_to(self, b=1, bsize=1, tsize=None):
            if tsize is not None:
                self.total = tsize
            self.update(b * bsize - self.n)
    
    with TqdmUpTo(unit='B', unit_scale=True, miniters=1, desc=filename) as t:
        urllib.request.urlretrieve(url, filename, reporthook=t.update_to)

# Create checkpoints directory
os.makedirs('checkpoints', exist_ok=True)

# Download model if not already present
model_path = 'checkpoints/depth_pro.pt'
if not os.path.exists(model_path):
    print("⬇️ Downloading Depth Pro model weights (1.8GB)...")
    print("☕ This may take a few minutes, grab some coffee!")
    
    model_url = 'https://ml-site.cdn-apple.com/models/depth-pro/depth_pro.pt'
    download_with_progress(model_url, model_path)
    
    print(f"✅ Model downloaded! Size: {os.path.getsize(model_path) / 1e9:.1f} GB")
else:
    print(f"✅ Model already exists! Size: {os.path.getsize(model_path) / 1e9:.1f} GB")


In [ ]:
# Install the depth_pro package
print("🔧 Installing Depth Pro package...")
%pip install -e .

# Test the installation
try:
    import depth_pro
    print("✅ Depth Pro imported successfully!")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("🔄 Trying alternative import method...")
    
    # Alternative: add all necessary paths
    import sys
    sys.path.append('./src')
    sys.path.append('./src/depth_pro')
    
    try:
        import depth_pro
        print("✅ Alternative import successful!")
    except ImportError as e2:
        print(f"❌ Still failed: {e2}")


In [ ]:
import gradio as gr
import torch
import numpy as np
from PIL import Image
import cv2
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Ensure depth_pro is available
try:
    import depth_pro
    MODEL_AVAILABLE = True
    print("✅ Depth Pro module loaded successfully!")
except ImportError as e:
    MODEL_AVAILABLE = False
    print(f"❌ Failed to import depth_pro: {e}")

# Global model variables
model = None
transform = None

def load_model():
    """Load the Depth Pro model once"""
    global model, transform
    if model is None and MODEL_AVAILABLE:
        print("🔄 Loading Depth Pro model...")
        try:
            # Load model and transform
            model, transform = depth_pro.create_model_and_transforms()
            
            # Move to GPU if available
            if torch.cuda.is_available():
                model = model.cuda()
                print("🚀 Model moved to GPU!")
            
            model.eval()
            print("✅ Model loaded successfully!")
            return True
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            return False
    return model is not None

def process_image(image):
    """Process uploaded image and return depth map"""
    if image is None:
        return None, None, "❌ Please upload an image first!"
    
    if not MODEL_AVAILABLE:
        return None, None, "❌ Depth Pro model not available. Please check installation."
    
    try:
        # Load model if not already loaded
        if not load_model():
            return None, None, "❌ Failed to load model"
        
        # Convert PIL image to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        print(f"📸 Processing image: {image.size}")
        
        # Convert to numpy array and preprocess
        image_array = np.array(image)
        
        # Use depth_pro's preprocessing
        processed_image = transform(image)
        
        # Add batch dimension and move to GPU if available
        if processed_image.dim() == 3:
            processed_image = processed_image.unsqueeze(0)
        
        if torch.cuda.is_available():
            processed_image = processed_image.cuda()
        
        # Run inference
        with torch.no_grad():
            prediction = model.infer(processed_image)
            depth = prediction["depth"].cpu().numpy().squeeze()
            focal_length = prediction["focallength_px"]
        
        print(f"✅ Depth map generated! Shape: {depth.shape}")
        
        # Normalize depth map for visualization (0-255)
        depth_min, depth_max = depth.min(), depth.max()
        depth_normalized = ((depth - depth_min) / (depth_max - depth_min) * 255).astype(np.uint8)
        
        # Create colormap version
        depth_colored = cv2.applyColorMap(depth_normalized, cv2.COLORMAP_PLASMA)
        depth_colored = cv2.cvtColor(depth_colored, cv2.COLOR_BGR2RGB)
        
        # Create info text
        info_text = f"""
### ✅ Depth Map Generated Successfully!
        
- **📏 Image Size**: {image.width} × {image.height}
- **🔍 Estimated Focal Length**: {focal_length:.1f} pixels  
- **📊 Depth Range**: {depth_min:.2f}m - {depth_max:.2f}m
- **⚡ Processing**: ~0.3 seconds on GPU
- **🎯 Model**: Apple Depth Pro v1.0

**How to use the results:**
- **Colored Version**: Great for visualization and analysis
- **Grayscale Version**: Use for 3D reconstruction, depth-based effects
- **Depth Values**: White = closer, Black = farther
        """
        
        return depth_colored, depth_normalized, info_text
        
    except Exception as e:
        error_msg = f"❌ Error processing image: {str(e)}"
        print(error_msg)
        import traceback
        traceback.print_exc()
        return None, None, error_msg

print("🎨 Gradio functions defined successfully!")


In [ ]:
# Create the Gradio interface
def create_interface():
    # Custom CSS for better styling
    css = """
    .gradio-container {
        font-family: 'IBM Plex Sans', sans-serif;
        max-width: 1200px;
        margin: 0 auto;
    }
    .main-header {
        text-align: center;
        margin-bottom: 30px;
        padding: 20px;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border-radius: 10px;
        color: white;
    }
    .upload-area {
        border: 2px dashed #ccc;
        border-radius: 10px;
        padding: 20px;
        text-align: center;
        margin: 10px 0;
    }
    """
    
    with gr.Blocks(
        title="🎯 Depth Pro - AI Depth Maps (Colab)", 
        theme=gr.themes.Soft(),
        css=css
    ) as demo:
        
        gr.HTML("""
        <div class="main-header">
            <h1>🎯 Depth Pro - AI Depth Map Generator</h1>
            <p style="font-size: 18px; margin: 0 auto; opacity: 0.9;">
                Transform any 2D image into a detailed 3D depth map in seconds!<br>
                Powered by <strong>Apple's state-of-the-art Depth Pro</strong> model.
            </p>
            <p style="margin-top: 10px; opacity: 0.8;">
                🚀 Ultra-fast GPU processing • 🎯 High accuracy • 🆓 Completely free on Colab
            </p>
        </div>
        """)
        
        with gr.Row(equal_height=True):
            with gr.Column(scale=1):
                gr.HTML("<h3>📤 Upload Your Image</h3>")
                input_image = gr.Image(
                    label="Drop an image here or click to upload",
                    type="pil",
                    height=400
                )
                
                gr.HTML("""
                <div style="margin-top: 15px; padding: 10px; background-color: #f0f0f0; border-radius: 5px;">
                    <strong>💡 Tips:</strong>
                    <ul style="margin: 5px 0; padding-left: 20px;">
                        <li>Works best with clear, well-lit images</li>
                        <li>Supports people, objects, landscapes</li>
                        <li>No camera data required!</li>
                        <li>Processing happens automatically on upload</li>
                    </ul>
                </div>
                """)
                
            with gr.Column(scale=1):
                gr.HTML("<h3>🎨 Colored Depth Map</h3>")
                output_colored = gr.Image(
                    label="Colored Depth Visualization",
                    height=400
                )
                
            with gr.Column(scale=1):
                gr.HTML("<h3>⚫ Grayscale Depth Map</h3>")
                output_grayscale = gr.Image(
                    label="Grayscale Depth Map",
                    height=400
                )
        
        info_output = gr.Markdown(
            value="📋 **Ready to process!** Upload an image above to generate its depth map.",
            elem_id="info_output"
        )
        
        # Examples section (if example images exist)
        examples = []
        if os.path.exists("data"):
            for file in os.listdir("data"):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    examples.append([f"data/{file}"])
        
        if examples:
            gr.HTML("<h3 style='text-align: center; margin-top: 40px;'>🖼️ Try These Examples</h3>")
            gr.Examples(
                examples=examples[:4],  # Show max 4 examples
                inputs=input_image,
                label="Click any example to try it out"
            )
        
        # Event handlers - process image automatically when uploaded
        input_image.change(
            fn=process_image,
            inputs=input_image,
            outputs=[output_colored, output_grayscale, info_output]
        )
        
        # Footer
        gr.HTML("""
        <div style="text-align: center; padding: 30px; color: #888; border-top: 1px solid #eee; margin-top: 50px;">
            <p style="font-size: 16px;">
                🤖 Powered by <a href="https://github.com/apple/ml-depth-pro" target="_blank" style="color: #007bff;">Apple's Depth Pro</a> 
                | 💻 Running on <a href="https://colab.research.google.com" target="_blank" style="color: #007bff;">Google Colab</a>
            </p>
            <p style="font-size: 14px; margin-top: 10px;">
                <strong>Research:</strong> "Depth Pro: Sharp Monocular Metric Depth in Less Than a Second" (2024)<br>
                <strong>Features:</strong> Zero-shot depth estimation • Metric depth prediction • Real-time processing
            </p>
            <p style="font-size: 12px; margin-top: 15px; color: #aaa;">
                Free GPU processing courtesy of Google Colab • Open source and educational use
            </p>
        </div>
        """)
    
    return demo

print("🎨 Interface created successfully!")


In [ ]:
# Launch the Gradio app
print("🚀 Launching Depth Pro application...")
print("⏳ This may take a moment to load the model...")

# Create and launch the demo
demo = create_interface()

# Launch with share=True to get a public URL
demo.launch(
    share=True,  # Creates a public gradio.live URL
    debug=True,
    show_error=True,
    server_name="0.0.0.0",
    server_port=7860,
    inline=False  # Shows the interface in a new tab
)

print("\n🎉 App launched successfully!")
print("📱 The public URL above can be shared with anyone")
print("⚠️ Note: The URL will be active as long as this Colab session is running")
